# Match - Minor random name differences

In [1]:
import pandas as pd
from pathlib import Path
from notebook_utils import finish_notebook

Get paths of input and output directories

In [2]:
input_data_dir = (Path.cwd().parent / "Mapping" / "Input" / "Flowlists").resolve()
existing_matches_dir = (Path.cwd().parent / "Mapping" / "Output" / "Mapped_files").resolve()

Read input dataframes

In [3]:
sp = pd.read_csv(input_data_dir / 'SimaProv9.4.csv')

In [4]:
ei = pd.read_csv(input_data_dir / 'ecoinventEFv3.7.csv')

[Stolen from https://github.com/jhuo2021/Importing_Agri-footprint_to_BW](https://github.com/jhuo2021/Importing_Agri-footprint_to_BW/blob/main/Data/migrations/agrifootprint-6-economic.json)

In [5]:
good_data = [
    ('2,4-D, dimethylamine salt', 'Dimethylamine'),
    ('Acids, unspecified', 'Acidity, unspecified'),
    ('Argon-40/kg', 'Argon-40'),
    ('Arsenic', 'Arsenic V'),
    ('Benzo(b,j,k)fluoranthene', 'Benzo(b)fluoranthene'),
    ('BOD5 (Biological Oxygen Demand)', 'BOD5, Biological Oxygen Demand'),
    ('BOD5 (Biological Oxygen Demand), NO', 'BOD5, Biological Oxygen Demand'),
    ('Chlorotoluron', 'Chlortoluron'),
    ('Chromium III', 'Chromium'),
    ('Coal, 26.4 MJ per kg', 'Coal, hard, unspecified, in ground'),
    ('Coal, 29.3 MJ per kg', 'Coal, hard, unspecified, in ground'),
    ('COD (Chemical Oxygen Demand)', 'COD, Chemical Oxygen Demand'),
    ('COD (Chemical Oxygen Demand), NO', 'COD, Chemical Oxygen Demand'),
    ('Dioxin, 2,3,7,8 Tetrachlorodibenzo-p-', 'Dioxins, measured as 2,3,7,8-tetrachlorodibenzo-p-dioxin'),
    ('Discarded fish, demersal', 'Discarded fish, demersal, to ocean'),
    ('Discarded fish, pelagic', 'Discarded fish, pelagic, to ocean'),
    ('Flurochloridone', 'Fluorochloridone'),
    ('Kaolin ore', 'Kaolinite, 24% in crude ore, in ground'),
    ('Lead-210/kg', 'Lead-210'),
    ('Metaldehyde (tetramer)', 'Metaldehyde'),
    ('Metolachlor, (S)', 'Metolachlor'),
    ('Nitrate compounds', 'Nitrate'),
    ('p-Xylene', 'Xylene'),
    ('Phenols, unspecified', 'Phenol'),
    ('Radioactive species, unspecified', 'Radioactive species, other beta emitters'),
    ('Radium-226/kg', 'Radium-226'),
    ('Radium-228/kg', 'Radium-228'),
    ('Sand, gravel and stone, extracted for use', 'Gravel, in ground'),
    ('Tri-allate', 'Triallate'),
]

# I am not sure that these are perfect...
unsure_data = [
    ('1-Butene', 'Butene'),
    ('2-Butene', 'Butene'),
    ('BTEX (Benzene, Toluene, Ethylbenzene, and Xylene), unspecified ratio', 'Benzene'),
    ('Calcium carbonate', 'Calcium, in ground'),
    ('Calcium chloride', 'Calcium, in ground'),
    ('Ethane, 1,1,2-trifluoro-, HFC-143', 'Ethane, 1,1,1-trifluoro-, HFC-143a'),
    ('Magnesium chloride', 'Magnesium, in ground'),
    ('Phosphate, NO', 'Phosphate'),
    ('Sulfur monoxide', 'Sulfur oxides'),
]

data = good_data + unsure_data

In [6]:
ecoinvent_names = pd.DataFrame(
    data,
    columns=["Flowable", "EcoinventFlowable"]
)

In [7]:
sp = sp.merge(ecoinvent_names, how="inner", on="Flowable")

In [ ]:
for c in ("air", "water", "soil", "resources"):
    sp_filtered = expand_simapro_context(sp, c)
    sp_filtered = add_ecoinvent_context_column(sp_filtered, label="EcoinventContext", kind=c)
    df = sp_filtered.merge(
        ei, 
        how="inner", 
        left_on=["EcoinventFlowable", "EcoinventContext"], 
        right_on=["Flowable", "Context"]
    )
    finish_notebook(
        df=df, 
        author="Chris Mutel",
        notebook_name="Match - Minor random name differences",
        filename=f'minor-name-differences-in-{c}'
    )

In [8]:
df = sp.merge(ei, how="inner", left_on="EcoinventFlowable", right_on="Flowable")
df

,Flowable_x,CAS No,Formula_x,Synonyms_x,Unit_x,Class_x,Context_x,Flow UUID,Description,EcoinventFlowable,...,Synonyms_y,Unit_y,Class_y,ExternalReference,Preferred,Context_y,FlowUUID,AltUnit,Unnamed: 11,Second CAS
0,Argon-40/kg,NaN,NaN,Argon-40/kg,kg,Raw materials,Raw materials,2726C3D7-C07B-4579-8B51-B833365E82F8,CAS number: 1290046-39-7,Argon-40,...,NaN,kg,chemical,NaN,NaN,air/unspecified,38321e66-f6ab-4908-b26f-c17267559bc5,NaN,NaN,13965-95-2; 1290046-39-7; 14163-25-8; 7440-37-1
1,Argon-40/kg,NaN,NaN,Argon-40/kg,kg,Raw materials,Raw materials,2726C3D7-C07B-4579-8B51-B833365E82F8,CAS number: 1290046-39-7,Argon-40,...,NaN,kg,chemical,NaN,NaN,natural resource/in air,8b46f615-69c9-4ca3-815e-a7bde116d202,NaN,NaN,13965-95-2; 1290046-39-7; 14163-25-8; 7440-37-1
2,Argon-40/kg,NaN,NaN,Argon-40/kg,kg,Airborne emissions,Airborne emissions,6EBC5693-ED93-481C-978A-CAA21B4A8430,CAS number: 1290046-39-7,Argon-40,...,NaN,kg,chemical,NaN,NaN,air/unspecified,38321e66-f6ab-4908-b26f-c17267559bc5,NaN,NaN,13965-95-2; 1290046-39-7; 14163-25-8; 7440-37-1
3,Argon-40/kg,NaN,NaN,Argon-40/kg,kg,Airborne emissions,Airborne emissions,6EBC5693-ED93-481C-978A-CAA21B4A8430,CAS number: 1290046-39-7,Argon-40,...,NaN,kg,chemical,NaN,NaN,natural resource/in air,8b46f615-69c9-4ca3-815e-a7bde116d202,NaN,NaN,13965-95-2; 1290046-39-7; 14163-25-8; 7440-37-1
4,Calcium carbonate,NaN,NaN,Calcium carbonate,kg,Raw materials,Raw materials,2EE55BBB-A073-4708-A157-925562703E5C,NaN,"Calcium, in ground",...,NaN,kg,geological,NaN,1.0,natural resource/in ground,c8fc4197-7410-42f2-aeb4-c08c6a693992,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Radium-228/kg,015262-20-1,NaN,Radium-228/kg,kg,Waterborne emissions,Waterborne emissions,CFA89026-8CD4-448A-8B23-F80507457053,Formula: Ra-228,Radium-228,...,NaN,kBq,chemical,NaN,NaN,water/ground-,fe807c9d-7e35-4500-9511-25b92cd6fe53,NaN,NaN,15262-20-1
323,Radium-228/kg,015262-20-1,NaN,Radium-228/kg,kg,Waterborne emissions,Waterborne emissions,CFA89026-8CD4-448A-8B23-F80507457053,Formula: Ra-228,Radium-228,...,NaN,kBq,chemical,NaN,NaN,"water/ground-, long-term",d4bb55d3-8165-466b-b502-d4b25fae104b,NaN,NaN,15262-20-1
324,Radium-228/kg,015262-20-1,NaN,Radium-228/kg,kg,Waterborne emissions,Waterborne emissions,CFA89026-8CD4-448A-8B23-F80507457053,Formula: Ra-228,Radium-228,...,NaN,kBq,chemical,NaN,NaN,water/ocean,09257283-72d9-4d13-838b-ee7f28e6d146,NaN,NaN,15262-20-1
325,Radium-228/kg,015262-20-1,NaN,Radium-228/kg,kg,Waterborne emissions,Waterborne emissions,CFA89026-8CD4-448A-8B23-F80507457053,Formula: Ra-228,Radium-228,...,NaN,kBq,chemical,NaN,NaN,water/surface water,d2312bc4-af11-445f-aa07-20d95b2794bd,NaN,NaN,15262-20-1


In [9]:
finish_notebook(
    df=df,
    author="Chris Mutel",
    notebook_name="Match - Minor land name differences",
    filename="minor-land-name-differences",
)